<a href="https://colab.research.google.com/github/nghiemkhoa1235-boop/B-i_t-p_AI/blob/main/Ch%E1%BB%89_tay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Import thư viện cần thiết
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import cv2
import os
from google.colab import drive, files
import zipfile
from PIL import Image
import matplotlib.pyplot as plt

# Mount Google Drive
drive.mount('/content/drive')

# 2. Kết nối và tải dữ liệu từ Google Drive
# Thay đổi đường dẫn theo thư mục của bạn
data_path = '/content/drive/MyDrive/ANN_NHOM 10/Palmistry-Đường Chỉ Tay/Train'  # Đường dẫn tới thư mục chứa ảnh

# 3. Tiền xử lý dữ liệu
def load_and_preprocess_data(data_path):
    images = []
    labels = []
    class_names = []

    if not os.path.isdir(data_path):
        print(f"Error: Data directory not found at {data_path}")
        return np.array([]), np.array([]), []

    # Lấy danh sách các thư mục con (classes)
    for class_idx, class_name in enumerate(sorted(os.listdir(data_path))):
        class_path = os.path.join(data_path, class_name)
        if os.path.isdir(class_path):
            class_names.append(class_name)

            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)

                # Đọc và xử lý ảnh
                img = cv2.imread(img_path)
                if img is not None:
                    # Chuyển sang ảnh xám
                    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    # Resize về 100x100
                    resized_img = cv2.resize(gray_img, (100, 100))
                    # Chuẩn hóa pixel về [0,1]
                    normalized_img = resized_img / 255.0

                    images.append(normalized_img)
                    labels.append(class_idx)

    return np.array(images), np.array(labels), class_names

# Tải và xử lý dữ liệu
X, y, class_names = load_and_preprocess_data(data_path)
print(f"Tổng số ảnh: {len(X)}")
print(f"Các class: {class_names}")

# Check if data was loaded before proceeding
if len(X) == 0:
    print("No data loaded. Please check the data_path.")
else:
    # 4. Chia dữ liệu train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Chuyển labels sang one-hot encoding
    y_train_cat = to_categorical(y_train, num_classes=3)
    y_test_cat = to_categorical(y_test, num_classes=3)

    print(f"Train samples: {len(X_train)}, Test samples: {len(X_test)}")

    # 5. Thiết kế mô hình ANN
    model = Sequential([
        Flatten(input_shape=(100, 100)),
        Dense(512, activation='relu'),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(3, activation='softmax')  # 3 classes
    ])

    # 6. Compile mô hình
    model.compile(
        optimizer=Adam(learning_rate=0.001),  # Tốc độ học nhanh hơn
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    # 7. Huấn luyện mô hình
    history = model.fit(
        X_train, y_train_cat,
        validation_data=(X_test, y_test_cat),
        epochs=50,
        batch_size=16,
        verbose=1
    )

    # 8. Đánh giá mô hình
    test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=0)
    print(f"Test Accuracy: {test_acc:.4f}")

    # Lưu mô hình
    model.save('/content/hand_gesture_model.h5')

    # Hàm dự đoán cho ảnh mới
    def predict_gesture(image_path, model, class_names):
        # Đọc và xử lý ảnh
        img = cv2.imread(image_path)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized_img = cv2.resize(gray_img, (100, 100))
        normalized_img = resized_img / 255.0

        # Reshape cho prediction
        img_array = normalized_img.reshape(1, 100, 100)

        # Dự đoán
        prediction = model.predict(img_array)
        predicted_class = np.argmax(prediction)
        confidence = np.max(prediction)

        return class_names[predicted_class], confidence

    # Hàm upload và dự đoán ảnh mới
    def upload_and_predict():
        uploaded = files.upload()

        for filename in uploaded.keys():
            print(f"Đang dự đoán cho ảnh: {filename}")

            # Dự đoán
            predicted_class, confidence = predict_gesture(filename, model, class_names)

            print(f"Kết quả dự đoán: {predicted_class}")
            print(f"Độ tin cậy: {confidence:.4f}")

            # Hiển thị ảnh
            img = Image.open(filename)
            plt.figure(figsize=(6, 6))
            plt.imshow(img)
            plt.title(f"Dự đoán: {predicted_class} (Tin cậy: {confidence:.2f})")
            plt.axis('off')
            plt.show()

            # Xóa file sau khi dự đoán
            os.remove(filename)

    print("\nMô hình đã được train xong!")
    print("Gọi upload_and_predict() để upload ảnh mới và dự đoán")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Tổng số ảnh: 236
Các class: ['Sinh_menh', 'Tinh_duyen', 'Tri_Tue']
Train samples: 188, Test samples: 48


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 10000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     5,120,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,285,123 (20.16 MB)

 Trainable params: 5,285,123 (20.16 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.4274 - loss: 3.5196 - val_accuracy: 0.6875 - val_loss: 0.7437
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.4359 - loss: 3.2064 - val_accuracy: 0.5208 - val_loss: 1.1495
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5336 - loss: 1.9398 - val_accuracy: 0.7292 - val_loss: 0.7213
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.4711 - loss: 2.0646 - val_accuracy: 0.5833 - val_loss: 0.9690
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5277 - loss: 1.2725 - val_accuracy: 0.4583 - val_loss: 0.8519
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.5900 - loss: 0.9981 - val_accuracy: 0.7708 - val_loss: 0.6638
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.5499 - loss: 0.9727 - val_accuracy: 0.7708 - val_loss: 0.6450
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.6751 - loss: 0.7285 - val_accuracy: 0.7917 - v

Test Accuracy: 0.9167

Mô hình đã được train xong!
Gọi upload_and_predict() để upload ảnh mới và dự đoán


In [4]:
# ===== GIAO DIỆN GRADIO SIÊU ĐẸP CHO MÔ HÌNH ANN NHẬN DIỆN CHỈ TAY =====
# 🌟 Code này chạy riêng biệt sau khi đã train xong model

# 1. Cài đặt và import thư viện
!pip install gradio

import gradio as gr
import numpy as np
import cv2
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import os

# 2. Load mô hình đã train
MODEL_PATH = '/content/hand_gesture_model.h5'

try:
    model = load_model(MODEL_PATH)
    print("✅ Model đã được load thành công!")
except Exception as e:
    print(f"❌ Lỗi khi load model: {e}")
    print("💡 Hãy đảm bảo bạn đã train model và lưu tại:", MODEL_PATH)

# 3. Định nghĩa class names (thay đổi theo dataset thực tế)
CLASS_NAMES = [
    "Chỉ tay chữ M",
    "Chỉ tay đuôi cá",
    "Chỉ tay mắt phượng"
]

# 4. Thông tin chi tiết về từng loại chỉ tay
PALMISTRY_INFO = {
    "Chỉ tay chữ M": """
🌟 **Chỉ tay chữ M - Người may mắn đặc biệt**

✨ **Đặc điểm nổi bật:**
• Có tài năng đặc biệt và thông minh vượt trội
• Trực giác tốt, khả năng lãnh đạo xuất sắc
• Thường đạt được thành công lớn trong sự nghiệp
• May mắn trong tình yêu và tài chính

🔮 **Vận mệnh:**
Bạn được sinh ra với vận may đặc biệt! Những đường chỉ tay tạo thành chữ M thể hiện sự kết hợp hoàn hảo giữa trí tuệ, trực giác và nghị lực. Cuộc đời bạn sẽ gặp nhiều cơ hội tốt và thành công vượt bậc.

🌟 **Lời khuyên:**
Hãy tin tưởng vào trực giác và không ngại đưa ra quyết định quan trọng!
    """,

    "Chỉ tay đuôi cá": """
🐠 **Chỉ tay đuôi cá - Người linh hoạt và thịnh vượng**

✨ **Đặc điểm nổi bật:**
• Linh hoạt và thích nghi tốt với mọi hoàn cảnh
• Khả năng kiếm tiền và tích lũy tài sản xuất sắc
• Thường gặp may mắn trong đầu tư và kinh doanh
• Có khả năng vượt qua khó khăn một cách sáng tạo

🔮 **Vận mệnh:**
Đường chỉ tay đuôi cá báo hiệu một tương lai thịnh vượng và giàu có. Bạn có thiên hướng kinh doanh tốt và khả năng nhìn xa trông rộng trong các quyết định tài chính.

🌟 **Lời khuyên:**
Hãy nắm bắt cơ hội đầu tư và tin tưởng vào năng lực kinh doanh của mình!
    """,

    "Chỉ tay mắt phượng": """
🦚 **Chỉ tay mắt phượng - Người có trí tuệ và địa vị**

✨ **Đặc điểm nổi bật:**
• Trí tuệ cao, khả năng nhìn xa trông rộng
• Thường được quý nhân phù trợ trong sự nghiệp
• Có khả năng lãnh đạo và ảnh hưởng lớn đến người khác
• Tương lai tươi sáng với địa vị cao trong xã hội

🔮 **Vận mệnh:**
Chỉ tay mắt phượng thể hiện sự cao quý và trí tuệ. Bạn được định sẵn cho những vị trí lãnh đạo và sẽ có ảnh hưởng lớn trong lĩnh vực mình theo đuổi.

🌟 **Lời khuyên:**
Phát huy khả năng lãnh đạo và không ngại thể hiện tầm nhìn độc đáo của mình!
    """
}

# 5. Hàm tiền xử lý ảnh
def preprocess_image(image):
    """Tiền xử lý ảnh giống như khi training"""
    if image is None:
        return None

    # Chuyển PIL Image sang numpy array
    if isinstance(image, Image.Image):
        image = np.array(image)

    # Chuyển sang grayscale nếu cần
    if len(image.shape) == 3:
        gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    else:
        gray_img = image

    # Resize về 100x100 như khi training
    resized_img = cv2.resize(gray_img, (100, 100))

    # Chuẩn hóa về [0,1]
    normalized_img = resized_img / 255.0

    # Reshape cho model (1, 100, 100)
    img_array = normalized_img.reshape(1, 100, 100)

    return img_array, resized_img

# 6. Hàm dự đoán chính
def predict_palmistry(image):
    """Hàm dự đoán loại chỉ tay"""

    if image is None:
        return None, "", "", "⚠️ Vui lòng tải lên một ảnh chỉ tay để phân tích!"

    try:
        # Tiền xử lý ảnh
        processed_img, resized_img = preprocess_image(image)

        if processed_img is None:
            return None, "Lỗi", "0%", "❌ Không thể xử lý ảnh này!"

        # Dự đoán với model
        predictions = model.predict(processed_img, verbose=0)

        # Lấy kết quả
        class_idx = np.argmax(predictions[0])
        confidence = predictions[0][class_idx] * 100

        predicted_class = CLASS_NAMES[class_idx]

        # Lấy thông tin chi tiết
        description = PALMISTRY_INFO.get(predicted_class, "Chưa có thông tin về loại chỉ tay này.")

        # Tạo ảnh processed để hiển thị
        processed_display = Image.fromarray((resized_img * 255).astype(np.uint8))

        return (
            processed_display,
            predicted_class,
            f"{confidence:.1f}%",
            description
        )

    except Exception as e:
        return None, "Lỗi", "0%", f"❌ Có lỗi xảy ra khi phân tích: {str(e)}"

# 7. Hàm reset giao diện
def reset_interface():
    """Reset toàn bộ giao diện"""
    return (
        None,  # input image
        None,  # processed image
        "",    # prediction
        "",    # confidence
        "👋 **Chào mừng bạn đến với ứng dụng bói chỉ tay AI!**\n\n✨ Hãy tải lên một ảnh chỉ tay rõ ràng để khám phá vận mệnh của bạn."
    )

# 8. CSS tùy chỉnh cho giao diện đẹp
custom_css = """
/* Import Google Fonts */
@import url('https://fonts.googleapis.com/css2?family=Orbitron:wght@400;700;900&family=Poppins:wght@300;400;600;700&display=swap');

/* Background và container chính */
.gradio-container {
    background: linear-gradient(135deg,
        #667eea 0%,
        #764ba2 25%,
        #f093fb 50%,
        #f5576c 75%,
        #4facfe 100%
    ) !important;
    background-size: 400% 400% !important;
    animation: gradientShift 15s ease infinite !important;
    min-height: 100vh !important;
}

@keyframes gradientShift {
    0% { background-position: 0% 50%; }
    50% { background-position: 100% 50%; }
    100% { background-position: 0% 50%; }
}

/* Title styling */
h1 {
    font-family: 'Orbitron', sans-serif !important;
    font-weight: 900 !important;
    font-size: 3rem !important;
    background: linear-gradient(45deg, #FFD700, #FFA500, #FF69B4, #9370DB) !important;
    -webkit-background-clip: text !important;
    -webkit-text-fill-color: transparent !important;
    background-clip: text !important;
    text-align: center !important;
    margin: 2rem 0 !important;
    text-shadow: 0 0 30px rgba(255, 215, 0, 0.3) !important;
    animation: titleGlow 3s ease-in-out infinite alternate !important;
}

@keyframes titleGlow {
    0% { filter: brightness(1); }
    100% { filter: brightness(1.3); }
}

/* Card và panel styling */
.gr-panel, .gr-box {
    background: rgba(255, 255, 255, 0.15) !important;
    backdrop-filter: blur(20px) !important;
    border: 1px solid rgba(255, 255, 255, 0.3) !important;
    border-radius: 25px !important;
    box-shadow:
        0 15px 35px rgba(0, 0, 0, 0.1),
        inset 0 1px 0 rgba(255, 255, 255, 0.3) !important;
    padding: 2rem !important;
    margin: 1rem !important;
    transition: all 0.3s ease !important;
}

.gr-panel:hover, .gr-box:hover {
    transform: translateY(-5px) !important;
    box-shadow:
        0 20px 40px rgba(0, 0, 0, 0.2),
        inset 0 1px 0 rgba(255, 255, 255, 0.3) !important;
}

/* Button styling siêu đẹp */
.gr-button {
    background: linear-gradient(45deg, #FF6B6B, #4ECDC4, #45B7D1, #96CEB4) !important;
    border: none !important;
    border-radius: 25px !important;
    color: white !important;
    font-weight: 700 !important;
    font-size: 1.1rem !important;
    padding: 15px 35px !important;
    box-shadow: 0 8px 25px rgba(0, 0, 0, 0.3) !important;
    transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1) !important;
    position: relative !important;
    overflow: hidden !important;
}

.gr-button::before {
    content: '' !important;
    position: absolute !important;
    top: 0 !important;
    left: -100% !important;
    width: 100% !important;
    height: 100% !important;
    background: linear-gradient(90deg, transparent, rgba(255, 255, 255, 0.3), transparent) !important;
    transition: left 0.5s !important;
}

.gr-button:hover::before {
    left: 100% !important;
}

.gr-button:hover {
    transform: translateY(-3px) scale(1.05) !important;
    box-shadow: 0 12px 35px rgba(0, 0, 0, 0.4) !important;
}

/* Input và textbox styling */
.gr-textbox, .gr-number, .gr-dropdown {
    background: rgba(255, 255, 255, 0.9) !important;
    border: 2px solid rgba(255, 215, 0, 0.3) !important;
    border-radius: 15px !important;
    color: #333 !important;
    font-family: 'Poppins', sans-serif !important;
    font-size: 1rem !important;
    padding: 1rem !important;
    transition: all 0.3s ease !important;
}

.gr-textbox:focus, .gr-number:focus, .gr-dropdown:focus {
    border-color: #FFD700 !important;
    box-shadow: 0 0 20px rgba(255, 215, 0, 0.4) !important;
    transform: scale(1.02) !important;
}

/* Image upload area */
.gr-image {
    border: 3px dashed rgba(255, 215, 0, 0.6) !important;
    border-radius: 20px !important;
    background: rgba(255, 255, 255, 0.1) !important;
    transition: all 0.3s ease !important;
}

.gr-image:hover {
    border-color: #FFD700 !important;
    background: rgba(255, 255, 255, 0.2) !important;
    transform: scale(1.02) !important;
}

/* Label styling */
label {
    color: white !important;
    font-weight: 600 !important;
    font-size: 1.1rem !important;
    text-shadow: 0 2px 4px rgba(0, 0, 0, 0.3) !important;
    margin-bottom: 0.5rem !important;
}

/* Markdown và text styling */
.gr-markdown {
    background: rgba(255, 255, 255, 0.1) !important;
    border-radius: 15px !important;
    padding: 1.5rem !important;
    color: white !important;
    font-family: 'Poppins', sans-serif !important;
    line-height: 1.8 !important;
    border: 1px solid rgba(255, 255, 255, 0.2) !important;
}

.gr-markdown h1, .gr-markdown h2, .gr-markdown h3 {
    color: #FFD700 !important;
    margin: 1rem 0 !important;
}

.gr-markdown strong {
    color: #FFA500 !important;
}

/* Responsive */
@media (max-width: 768px) {
    h1 {
        font-size: 2rem !important;
    }

    .gr-button {
        padding: 12px 25px !important;
        font-size: 1rem !important;
    }

    .gr-panel, .gr-box {
        padding: 1rem !important;
        margin: 0.5rem !important;
    }
}

/* Animation cho các elements */
.gr-panel, .gr-box, .gr-button, .gr-textbox, .gr-image {
    animation: fadeInUp 0.6s ease-out !important;
}

@keyframes fadeInUp {
    from {
        opacity: 0;
        transform: translateY(30px);
    }
    to {
        opacity: 1;
        transform: translateY(0);
    }
}

/* Hiệu ứng sparkle */
@keyframes sparkle {
    0%, 100% { opacity: 0; transform: scale(0); }
    50% { opacity: 1; transform: scale(1); }
}

/* Custom scrollbar */
::-webkit-scrollbar {
    width: 12px;
}

::-webkit-scrollbar-track {
    background: rgba(255, 255, 255, 0.1);
    border-radius: 10px;
}

::-webkit-scrollbar-thumb {
    background: linear-gradient(45deg, #FFD700, #FFA500);
    border-radius: 10px;
}

::-webkit-scrollbar-thumb:hover {
    background: linear-gradient(45deg, #FFA500, #FF69B4);
}
"""

# 9. Tạo giao diện Gradio siêu đẹp
def create_interface():
    with gr.Blocks(
        css=custom_css,
        title="🔮 Bói Chỉ Tay AI - Khám Phá Vận Mệnh",
        theme=gr.themes.Soft(
            primary_hue="orange",
            secondary_hue="purple",
            neutral_hue="slate"
        )
    ) as interface:

        # Header
        gr.HTML("""
        <div style="text-align: center; padding: 2rem 0;">
            <h1>🔮 BÓI CHỈ TAY BẰNG AI ✋</h1>
            <p style="font-size: 1.4rem; color: rgba(255, 255, 255, 0.9); font-weight: 300; text-shadow: 0 2px 4px rgba(0,0,0,0.3); font-family: 'Poppins', sans-serif;">
                ✨ Khám phá vận mệnh qua đường chỉ tay với công nghệ Neural Network ✨
            </p>
            <div style="width: 100px; height: 4px; background: linear-gradient(45deg, #FFD700, #FFA500); margin: 1rem auto; border-radius: 2px;"></div>
        </div>
        """)

        # Main interface
        with gr.Row():
            # Left column - Input
            with gr.Column(scale=1):
                gr.Markdown("### 📸 **Tải ảnh chỉ tay của bạn**")

                input_image = gr.Image(
                    type="pil",
                    label="🖼️ Chọn ảnh chỉ tay rõ ràng",
                    height=400,
                    elem_id="input_image"
                )

                with gr.Row():
                    predict_btn = gr.Button(
                        "🔮 Phân tích vận mệnh",
                        variant="primary",
                        scale=2,
                        elem_id="predict_button"
                    )

                    reset_btn = gr.Button(
                        "🔄 Làm mới",
                        variant="secondary",
                        scale=1,
                        elem_id="reset_button"
                    )

            # Right column - Results
            with gr.Column(scale=1):
                gr.Markdown("### 🔍 **Ảnh đã xử lý**")

                processed_image = gr.Image(
                    label="Ảnh grayscale 100x100",
                    height=200,
                    elem_id="processed_image"
                )

                gr.Markdown("### 🎯 **Kết quả dự đoán**")

                with gr.Row():
                    prediction_text = gr.Textbox(
                        label="🏷️ Loại chỉ tay",
                        placeholder="Chưa phân tích",
                        interactive=False,
                        scale=2
                    )

                    confidence_text = gr.Textbox(
                        label="📊 Độ chính xác",
                        placeholder="0%",
                        interactive=False,
                        scale=1
                    )

        # Description area
        gr.Markdown("### 📖 **Giải thích chi tiết về vận mệnh**")

        description_output = gr.Markdown(
            value="👋 **Chào mừng bạn đến với ứng dụng bói chỉ tay AI!**\n\n✨ Hãy tải lên một ảnh chỉ tay rõ ràng để khám phá vận mệnh của bạn.",
            elem_id="description_area"
        )

        # Footer
        gr.HTML("""
        <div style="text-align: center; margin-top: 3rem; padding: 2rem; border-top: 1px solid rgba(255, 255, 255, 0.2);">
            <p style="color: rgba(255, 255, 255, 0.8); font-style: italic; margin-bottom: 1rem;">
                ⚠️ <strong>Lưu ý:</strong> Đây chỉ là ứng dụng giải trí sử dụng AI. Kết quả chỉ mang tính chất tham khảo!
            </p>
            <p style="color: rgba(255, 255, 255, 0.6); font-size: 0.9rem;">
                Được phát triển với ❤️ bằng TensorFlow & Gradio • Neural Network Model
            </p>
            <div style="margin-top: 1rem;">
                <span style="font-size: 1.5rem;">🔮 ✨ 🌟 ⭐ 💫</span>
            </div>
        </div>
        """)

        # Event handlers
        predict_btn.click(
            fn=predict_palmistry,
            inputs=[input_image],
            outputs=[processed_image, prediction_text, confidence_text, description_output],
            api_name="predict"
        )

        reset_btn.click(
            fn=reset_interface,
            inputs=None,
            outputs=[input_image, processed_image, prediction_text, confidence_text, description_output]
        )

        # Auto-predict khi upload ảnh
        input_image.change(
            fn=predict_palmistry,
            inputs=[input_image],
            outputs=[processed_image, prediction_text, confidence_text, description_output]
        )

    return interface

# 10. Chạy ứng dụng
if __name__ == "__main__":
    print("🚀 Đang khởi động ứng dụng bói chỉ tay AI...")
    print(f"📁 Model path: {MODEL_PATH}")
    print(f"🏷️ Classes: {CLASS_NAMES}")
    print("="*50)

    # Tạo và chạy giao diện
    app = create_interface()

    # Launch với cấu hình tối ưu
    app.launch(
        share=True,           # Tạo public link
        debug=False,          # Tắt debug để giao diện đẹp hơn
        show_error=True,      # Hiển thị lỗi nếu có
        server_name="0.0.0.0", # Cho phép truy cập từ bên ngoài
        server_port=7860,     # Port mặc định
        inbrowser=True,       # Tự động mở browser
        favicon_path=None,    # Có thể thêm favicon
        auth=None,           # Có thể thêm authentication
        max_threads=10       # Số thread tối đa
    )

✅ Model đã được load thành công!
🚀 Đang khởi động ứng dụng bói chỉ tay AI...
📁 Model path: /content/hand_gesture_model.h5
🏷️ Classes: ['Chỉ tay chữ M', 'Chỉ tay đuôi cá', 'Chỉ tay mắt phượng']
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c4ebb17160b055427e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
